<p style="text-align: center; color: #D6A4FF; font-family: 'Montserrat'; font-weight: bold; font-size: 36px">Notebook de exemplo para como corretamente fazer o ETL dos dados</p>

<ul style="font-family: 'Montserrat'; color: white; background-color: #FFFFF; font-size: 22px; padding: 5px; border-radius: 5px;">
  <li>Apresenta apenas como organizar os dados com um exemplo em JSON.</li>
  <li>Quais devem ser os atributos para envio no Postgre DB.</li>
</ul>

In [65]:
import pandas as pd
import numpy as np
import os, json
import itertools

<p style="text-align: center; color: #D6A4FF; font-family: 'Montserrat'; font-weight: bold; font-size: 22px;">Exemplo Calendar API</p>

<ul style="font-family: 'Montserrat'; color: white; background-color: #FFFFF; font-size: 16px; padding: 5px; border-radius: 5px;">
  <li>Como coletar os dados das interações dos participantes em uma meeting</li>
  <li>Como criar as interações entre os participantes de cada meeting dos dados extraídos</li>
</ul>
<br>
Pontos de atenção
<ul style="font-family: 'Montserrat'; color: white; background-color: #FFFFF; font-size: 16px; padding: 5px; border-radius: 5px;">
  <li>O sender e receiver devem manter essa nomeclatura mesmo quando o sender é um participante da reunião. Precisamos criar as interações entre cada colaborador</li>
  <li>Esse código é apenas um exemplo, mas os resultados devem estar exatamente iguais para garantirmos a replicação correta dos dados</li>
</ul>


In [66]:
# Importando os dados de exemplo do Calendar API
# Obviamente esses dados serão extraídos diretamente da chamada da Calendar API, aqui é apenas um exemplo

with open('calendar_data_example.json', 'r') as file:
    # Coletando os dados
    # No seu caso aqui ao inves de json.load, teríamos o .get da API do Calendar API.
    data = json.load(file)
    result_calendar = []
    # Aqui acessamos os dados de calendário de cada usuário que extraímos.
    # No seu caso, você vai estar coletando o nome de cada usuário e extraindo seu calendário
    for data_item in data['calendars']:
        # Aqui nos coletamos os items/meetings dentro da agenda do nosso colaborador, cada meeting corresponde ao período que você utilizou quando requisitou os dados no .get()
        for meeting in data_item['items']:
            # Aqui nos coletamos todos os participantes da agenda e adicionamos quem enviou a agenda, precisamos utilizar o set() para eliminar duplicatas
            users_agenda = set([person['email'] for person in meeting['attendees']] + [data_item['summary']])
            # Aqui nos realizamos a combinação de cada participante 2 a 2, ou seja podemos ter: anfitrião > participante, como participante > participante
            # Isso é muito importante para garantir os dados de interação de cada pessoa da reunião.
            combinations = list(itertools.combinations(users_agenda, 2))
            # Depois disso, realizamos o processo de salvar o resultado.
            result_calendar.extend([
                {
                 'sender':dupla[0],
                 'receiver':dupla[1],
                 'data': pd.Timestamp(meeting['start']['dateTime']).date(),
                 'de_onde':'calendar_api'
                }
                for dupla in combinations
            ])

# Resultado da extração de 2 reuniões.
result_calendar = pd.DataFrame(result_calendar)

<p style="text-align: center; color: #D6A4FF; font-family: 'Montserrat'; font-weight: bold; font-size: 22px;">Exemplo extração dos dados Calendar API [CHAT-GPT]</p>
<ul style="font-family: 'Montserrat'; color: white; background-color: #FFFFF; font-size: 16px; padding: 5px; border-radius: 5px;">
  <li>Você pode adaptar o seu codigo conforme o que o chatgpt apresenta abaixo.</li>
  <li>O resultado do código acima só precisar ser igual ao o que você implementar.</li>
</ul>

In [ ]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
from datetime import datetime, timedelta
import pytz

# Path to your service account JSON
SERVICE_ACCOUNT_FILE = 'path/to/your-service-account.json'

# Scopes required (read-only access)
SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']

# Admin email to impersonate (must be a Google Workspace admin)
DELEGATED_USER = 'admin@yourdomain.com'

# Authenticate with service account & impersonate admin
credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES
)
delegated_credentials = credentials.with_subject(DELEGATED_USER)

# Initialize Google Calendar API client
calendar_service = build('calendar', 'v3', credentials=delegated_credentials)

# Function to fetch meetings for a given user
def get_user_meetings(user_email, days_back=7, days_forward=7):
    # Define time range (past and future days)
    tz = pytz.UTC
    now = datetime.utcnow().replace(tzinfo=tz)
    start_time = now - timedelta(days=days_back)  # Past events
    end_time = now + timedelta(days=days_forward)  # Future events

    events_result = calendar_service.events().list(
        calendarId=user_email,
        timeMin=start_time.isoformat(),
        timeMax=end_time.isoformat(),
        singleEvents=True,
        orderBy='startTime'
    ).execute()

    events = events_result.get('items', [])
    
    print(f"\nMeetings for {user_email} from {start_time.date()} to {end_time.date()}:")
    print("=" * 50)

    for event in events:
        start_time = event.get('start', {}).get('dateTime', event.get('start', {}).get('date'))
        organizer = event.get('organizer', {}).get('email', 'Unknown')
        attendees = [attendee['email'] for attendee in event.get('attendees', []) if 'email' in attendee]

        print(f"Meeting: {event.get('summary', 'No Title')}")
        print(f"Organizer: {organizer}")
        print(f"Attendees: {attendees}")
        print(f"Start Time: {start_time}")
        print("-" * 50)

# List of users in the company (you can fetch this dynamically)
users = ['user1@yourdomain.com', 'user2@yourdomain.com']

for user in users:
    get_user_meetings(user, days_back=7, days_forward=7)  # Past & upcoming week

<p style="text-align: center; color: #D6A4FF; font-family: 'Montserrat'; font-weight: bold; font-size: 22px;">Exemplo Gmail API</p>

<ul style="font-family: 'Montserrat'; color: white; background-color: #FFFFF; font-size: 16px; padding: 5px; border-radius: 5px;">
  <li>Como coletar os dados das interações via email dos colaboradores</li>
  <li>Como juntar esses dados com o que foi coletado do Calendar API anteriormente</li>
</ul>

In [63]:
# Importando os dados de exemplo do GMAIL API
# Obviamente esses dados serão extraídos diretamente da chamada da GMAIL API, aqui é apenas um exemplo

with open('gmail_data_example.json', 'r') as file:
    # Como anteriormente, "coletando" os dados de maneira manual
    data = json.load(file)
    result_gmail = []
    # Iterando sobre cada email, aqui nós criamos as interações entre todas as pessoas que participaram do email, independente se foi o indivíduo que enviou o email
    for email in data['messages']:
        # Data do envio do email sendo coletado através do atributo messages
        data_email = [conteudo['value'] for conteudo in email['payload']['headers'] if conteudo['name'] == 'Date'][0]

        # Aqui coletamos os emails, tem casos em que os colaboradores em cópia são mais de um, como Cc: "user@, user2@" veja como não é uma lista, por isso utilizamos o split
        lista_emails = set([
            email.strip()
            for conteudo in email['payload']['headers'] if conteudo['name'] != 'Date'
            for email in conteudo['value'].split(',')
        ])

        # Criando todas as combinações possíveis não repetidas dos emails citados no envio
        combinations = list(itertools.combinations(lista_emails, 2))
        result_gmail.extend([
            {
                'sender':dupla[0],
                'receiver':dupla[1],
                'data': pd.Timestamp(data_email).date(),
                'de_onde':'gmail_api'
            }
            for dupla in combinations
        ])

result_gmail = pd.DataFrame(result_gmail)

<p style="text-align: center; color: #D6A4FF; font-family: 'Montserrat'; font-weight: bold; font-size: 22px;">Realizando o Join final e subindo os dados</p>

<ul style="font-family: 'Montserrat'; color: white; background-color: #FFFFF; font-size: 16px; padding: 5px; border-radius: 5px;">
  <li> Utilize da função abaixo para criar o hashmap de iteração entre os colaboradores</li>
  <li>Para finalizar, basta juntar ambos os dataframes, selecionar as colunas sender, receiver e data e fazer o join com a base de informação do seu RH e enviar ao nosso Oracle</li>
</ul>

In [71]:
df_final = pd.concat([result_calendar, result_gmail], axis=0)

# Utilize dessa função para criar o hashmap dos colaboradores da organização
import hashlib
def generate_hash(sender, receiver):
    interaction = f"{min(sender, receiver)}-{max(sender, receiver)}"  # Ensure order consistency
    return hashlib.sha256(interaction.encode()).hexdigest()


df_final['interaction_id'] = df_final.apply(lambda row: generate_hash(row['sender'], row['receiver']), axis=1) ; df_final

,sender,receiver,data,de_onde,interaction_id
0,user4@company.com,user3@company.com,2025-03-05,calendar_api,61da940bf15029fa40308daf5c1561bc85fd5a5d14f1b9...
1,user4@company.com,user2@company.com,2025-03-05,calendar_api,2c6c91b40e8067ee83c433e700fb12f794843652a2d4a0...
2,user4@company.com,user1@company.com,2025-03-05,calendar_api,78bd6cde6e1080e1a0aec4d70099aecdef5cecb0213596...
3,user4@company.com,user5@company.com,2025-03-05,calendar_api,a2967b493347c2af442baf8455c1cf1617f1798b43c02a...
4,user3@company.com,user2@company.com,2025-03-05,calendar_api,bccd40a33ca2d17f86a999dc6fcb17475e351a2abe1cd4...
5,user3@company.com,user1@company.com,2025-03-05,calendar_api,c70f788284647680a60dd030bec8f69fcd9438fc1b9bd3...
6,user3@company.com,user5@company.com,2025-03-05,calendar_api,683e0c1d2b6e261d07b310ce024131bde9c96927afb41f...
7,user2@company.com,user1@company.com,2025-03-05,calendar_api,aedf17d1a4886adc4ac119c3c24b60bf4a094860cfa92d...
8,user2@company.com,user5@company.com,2025-03-05,calendar_api,42b964b342866bbbfc78daa34724128ebbe185a7e7badb...
9,user1@company.com,user5@company.com,2025-03-05,calendar_api,73c44ef17a2088e45e038792c81127c5a39e71dbe116f7...
